In [5]:
import pandas as pd
from collections import Counter
import os
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
import numpy as np
from scipy import stats


data_dir = f'data/new_tests'
project = '2022-09-12_15_30_51-modified'
dir = f'{data_dir}/{project}'

df = pd.DataFrame()
for file in os.listdir(dir):
    if file.startswith('Session'):
        with open(f'{dir}/{file}') as f:
            starttime_millis = int(f.readline().rstrip()[17:])
        session = pd.read_csv(f'{dir}/{file}', skiprows=1)
    else:
        tmp = pd.read_csv(f'{dir}/{file}', skiprows=1)
        df = pd.concat([df, tmp], ignore_index=True)
df = df.sort_values(by=['timestamp'], ignore_index=True)

df = df.loc[:len(df)/2, :]


print(len(df))
# get readable time (delta from start time)
real_times = []
for time_millis in df['real time']:
    delta = (time_millis - starttime_millis) # convert to seconds
    real_times.append(str(timedelta(milliseconds=delta)))
df['human time'] = real_times


# get time delta at each sample
deltas = []
for i, timestamp in enumerate(df['timestamp']):
    if i == 0:
        deltas.append(np.nan)
        continue
    # get time since last sample, convert to Hz
    deltas.append( (df['timestamp'][i] - df['timestamp'][i-1]) / 1e9)
df['delta'] = deltas

df.to_csv(f'{data_dir}/processed/{project}-processed.csv')

print(df['delta'].describe())
print(df)
import plotly.express as px

fig = px.line(df, x='human time', y='delta', title='Time delta', 
                labels={'human time': 'Time', 
                        'delta': 'Time delta (s)'})
fig.show(renderer='browser')


2022-09-12_15_30_51.1.csv
Session.2022-09-12_15_30_51.csv
2022-09-12_15_30_51.0.csv
35946
count    35945.000000
mean         3.391445
std        439.852970
min          0.006012
25%          0.992584
50%          0.992706
75%          0.992798
max      83381.998268
Name: delta, dtype: float64
             timestamp     acc_x     acc_y     acc_z      real time activity  \
0      262138829750576  9.880524 -0.253311  1.163818  1663058735524    Other   
1      262139822792568  9.880661 -0.253464  1.163971  1663058735808    Other   
2      262140815773525  9.882477 -0.254456  1.173248  1663058736840    Other   
3      262141808754482  9.883636 -0.253647  1.175690  1663058737571    Other   
4      262142801643886  9.885452 -0.254272  1.177231  1663058738767    Other   
...                ...       ...       ...       ...            ...      ...   
35941  384040357420907  9.874908 -0.256516  1.157547  1663049649483    Other   
35942  384041350310311  9.878174 -0.255569  1.156876  166304964948